# ANALYSIS
TREND 1:Comparing the performance of school Types, charter schools performs better than District School.
TREND 2:There is negative correlation on school performances between total school budget and average sores
TREND 3:The size of a school also have a great influence on students scores. The smaller the class (<1000) 
the higher their overall passing Rate.                                                                                      

In [1]:
#import all dependencies 
import pandas as pd
import os
import numpy as np

In [2]:
#import files from files source
schools_file = os.path.join("schools_complete.csv")
students_file = os.path.join("students_complete.csv")

In [6]:
#Reading files into dataFrame
schools_df = pd.read_csv(schools_file)
students_df = pd.read_csv(students_file)


In [8]:
#Change the 'name' column of schools_df to 'school'
schools_df = schools_df.rename(columns={"name":"school"})

# District Summary

In [9]:
'''
District Summary
Create high-level snapshot (in table form) of district's key metrics, including:
- total schools
- total students (formatted with thousands separators)
- total budget (formatted as currency)
- average math score 
- average reading score
- % passing math
- % passing reading
- overall passing rate (average of % passing math & % passing reading)
'''

"\nDistrict Summary\nCreate high-level snapshot (in table form) of district's key metrics, including:\n- total schools\n- total students (formatted with thousands separators)\n- total budget (formatted as currency)\n- average math score \n- average reading score\n- % passing math\n- % passing reading\n- overall passing rate (average of % passing math & % passing reading)\n"

In [12]:
#calculate total number of students by counting student ids
#use a variable instead of direct calculation because it is needed to determine 
#percentage of students passing reading and math

total_students = students_df['Student ID'].count()
total_students

39170

In [14]:
#calculate number of students with passing math scores -- create data frame with passing scores,
passing_math = students_df.loc[students_df['math_score'] >= 70, ['math_score']]
pct_passing_math = (passing_math['math_score'].count() / total_students) * 100
pct_passing_math

74.9808526933878

In [15]:
#calculate number of students with passing read scores -- create data frame with passing scores,
passing_reading = students_df.loc[students_df['reading_score'] >= 70, ['reading_score']]
pct_passing_reading = (passing_reading['reading_score'].count() / total_students) * 100


In [16]:
#calculate number of students with passing read scores -- create data frame with passing scores,
passing_reading = students_df.loc[students_df['reading_score'] >= 70, ['reading_score']]
pct_passing_reading = (passing_reading['reading_score'].count() / total_students) * 100


In [18]:
# Average of % passing reading and % passing match
overall_passing_rate = ((pct_passing_math + pct_passing_reading) / 2)
overall_passing_rate


80.39315802910392

In [19]:
# create district_summary dataframe for display
district_summary = pd.DataFrame({
    'Total Schools': schools_df['School ID'].count(),
    'Total Students': [total_students],
    'Total Budget': schools_df['budget'].sum(),
    'Average Math Score': students_df['math_score'].mean(),
    'Average Reading Score': students_df['reading_score'].mean(),
    '% Passing Math': [pct_passing_math],
    '% Passing Reading': [pct_passing_reading],
    'Overall Passing Rate': [overall_passing_rate]})

In [20]:
# format 'Total Students' with thousands separator and 'Total Budget' with currency and thousands sep
district_summary['Total Students'] = district_summary['Total Students'].map('{:,}'.format)
district_summary['Total Budget'] = district_summary['Total Budget'].map('${:,.2f}'.format)

#reassemble summary columns in display order
district_summary = district_summary[['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score',
                                    'Average Reading Score', '% Passing Math', '% Passing Reading', 
                                    'Overall Passing Rate']]

#show summary
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


# School Summary

In [21]:
'''
School Summary
Create overview table that summarizes key metrics about each school:
- School name
- School type (Charter / District)
- Total school budget
- Budget per student
- Average math score
- Average reading score
- % passing math
- % passing reading
- Overall passing rate (average of % passing math & % passing reading)
'''

'\nSchool Summary\nCreate overview table that summarizes key metrics about each school:\n- School name\n- School type (Charter / District)\n- Total school budget\n- Budget per student\n- Average math score\n- Average reading score\n- % passing math\n- % passing reading\n- Overall passing rate (average of % passing math & % passing reading)\n'

In [22]:
# create dataframe containing only students who pass math
students_passing_math = students_df.loc[students_df['math_score'] > 70]

In [24]:
# reduce dataframe to include only high school name and a new field that will be used by the groupby for the 
students_passing_math_red = pd.DataFrame({'school': students_passing_math['school'],
                                         'nbr_students_passing_math': 0})

In [25]:
# group students passing math by high school and count. The count of students in the students_passing_math_red 
#Dataset will go in the nbr_students_passing_math column.
students_passing_math_by_school = students_passing_math_red.groupby(['school']).count().reset_index()


In [26]:
# create dataframe containing only students who pass reading
students_passing_reading = students_df.loc[students_df['reading_score'] > 70]

In [90]:
# reduce dataframe to include only high school and a new, empty field to contain the student counter created
# by the groupby in the next step.
students_passing_reading_red = pd.DataFrame({'school': students_passing_reading['school'],
                                          'nbr_students_passing_reading': 0})

In [91]:
# group students passing reading by high school and count, reset index so all columns can be used for processing
students_passing_reading_by_school = students_passing_reading_red.groupby(['school']).count().reset_index()

In [92]:
#group students by school to calculate average math and reading scores per school
school_group = students_df.groupby(['school'])

In [30]:
# build dataframe with calculated average score. reset the index so all columns can be used for processing
avg_scores_df = pd.DataFrame({'avg_math_score': school_group['math_score'].mean(),
                              'avg_reading_score': school_group['reading_score'].mean()}).reset_index()

In [31]:
# create 'super table' of schools_df, students passing reading, students passing math and average math scores.
super_school_df = pd.merge(schools_df, avg_scores_df, on = 'school') \
.merge(students_passing_math_by_school, on = 'school') \
.merge(students_passing_reading_by_school, on = 'school') 

In [32]:
pct_students_passing_math_school = (super_school_df['nbr_students_passing_math'] / super_school_df['size']) * 100

In [33]:
#calculate percentage of students passing reading
pct_students_passing_reading_school = (super_school_df['nbr_students_passing_reading'] / \
                                      super_school_df['size']) * 100


In [34]:
#calculate overall passing rate
overall_passing_rate_school = ((pct_students_passing_math_school + pct_students_passing_reading_school) / 2)

In [37]:
# create district_summary dataframe for display
school_summary_df = pd.DataFrame({'School Name': super_school_df['school'],
                               'School Type': super_school_df['type'],
                               'Total Students': super_school_df['size'],
                               'Total School Budget': super_school_df['budget'],
                               'Per Student Budget': (super_school_df['budget'] / super_school_df['size']),
                               'Average Math Score': super_school_df['avg_math_score'],
                               'Average Reading Score': super_school_df['avg_reading_score'],
                               '% Passing Math': pct_students_passing_math_school,
                               '% Passing Reading': pct_students_passing_reading_school,
                               'Overall Passing Rate': overall_passing_rate_school})
school_summary_df.head()

,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Passing Rate,Per Student Budget,School Name,School Type,Total School Budget,Total Students
0,63.318478,78.813850,76.629414,81.182722,71.066164,655.0,Huang High School,District,1910635,2917
1,63.750424,78.433367,76.711767,81.158020,71.091896,639.0,Figueroa High School,District,1884411,2949
2,89.892107,92.617831,83.359455,83.725724,91.254969,600.0,Shelton High School,Charter,1056600,1761
3,64.746494,78.187702,77.289752,80.934412,71.467098,652.0,Hernandez High School,District,3022020,4635
4,89.713896,93.392371,83.351499,83.816757,91.553134,625.0,Griffin High School,Charter,917500,1468


In [38]:
#format 'Total School Budget' & 'Per Student Budget' as currency
school_summary_df['Total School Budget'] = school_summary_df['Total School Budget'].map('${:,.2f}'.format)
school_summary_df['Per Student Budget'] = school_summary_df['Per Student Budget']

In [39]:
#Reassemble columns in display order, reset index to 'School Name' and remove index label.
school_summary_df = school_summary_df[['School Name', 'School Type', 'Total Students', 'Total School Budget',
                                 'Per Student Budget', 'Average Math Score', 'Average Reading Score',
                                 '% Passing Math', '% Passing Reading', 'Overall Passing Rate']] \
                 .set_index('School Name').rename_axis(None)

# display school summary
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Huang High School,District,2917,"$1,910,635.00",655.0,76.629414,81.182722,63.318478,78.813850,71.066164
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.711767,81.158020,63.750424,78.433367,71.091896
Shelton High School,Charter,1761,"$1,056,600.00",600.0,83.359455,83.725724,89.892107,92.617831,91.254969
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.289752,80.934412,64.746494,78.187702,71.467098
Griffin High School,Charter,1468,"$917,500.00",625.0,83.351499,83.816757,89.713896,93.392371,91.553134
Wilson High School,Charter,2283,"$1,319,574.00",578.0,83.274201,83.989488,90.932983,93.254490,92.093736
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.061895,83.975780,89.558665,93.864370,91.711518
Bailey High School,District,4976,"$3,124,928.00",628.0,77.048432,81.033963,64.630225,79.300643,71.965434
Holden High School,Charter,427,"$248,087.00",581.0,83.803279,83.814988,90.632319,92.740047,91.686183
Pena High School,Charter,962,"$585,858.00",609.0,83.839917,84.044699,91.683992,92.203742,91.943867


# Top Performing School

In [40]:
'''
Top Performing Schools (By Passing Rate)
Highlight the 5 top performing schools based on Overall Passing Rate
- School Name
- School type (Charter / District)
- Total students
- Total school budget
- Budget per student
- Average Math, Reading Scores
- % Passing Math and Reading (score > 70)
- Overall Passing Rate
'''

'\nTop Performing Schools (By Passing Rate)\nHighlight the 5 top performing schools based on Overall Passing Rate\n- School Name\n- School type (Charter / District)\n- Total students\n- Total school budget\n- Budget per student\n- Average Math, Reading Scores\n- % Passing Math and Reading (score > 70)\n- Overall Passing Rate\n'

In [43]:
# use nlargest on school summary created in previous step to find top 5
top_performing_schools_df = school_summary_df.nlargest(5, 'Overall Passing Rate')


In [44]:
#show dataframe
top_performing_schools_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Wilson High School,Charter,2283,"$1,319,574.00",578.0,83.274201,83.989488,90.932983,93.254490,92.093736
Pena High School,Charter,962,"$585,858.00",609.0,83.839917,84.044699,91.683992,92.203742,91.943867
Wright High School,Charter,1800,"$1,049,400.00",583.0,83.682222,83.955000,90.277778,93.444444,91.861111
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.061895,83.975780,89.558665,93.864370,91.711518
Holden High School,Charter,427,"$248,087.00",581.0,83.803279,83.814988,90.632319,92.740047,91.686183


# Bottom Performing Schools (By Passing Rate)

In [45]:
'''
Bottom Performing Schools (By Passing Rate)
Highlight the 5 worst performing schools based on Overall Passing Rate
- School Name
- School type (Charter / District)
- Total students
- Total school budget
- Budget per student
- Average Math, Reading Scores
- % Passing Math and Reading (score > 70)
- Overall Passing Rate
'''

'\nBottom Performing Schools (By Passing Rate)\nHighlight the 5 worst performing schools based on Overall Passing Rate\n- School Name\n- School type (Charter / District)\n- Total students\n- Total school budget\n- Budget per student\n- Average Math, Reading Scores\n- % Passing Math and Reading (score > 70)\n- Overall Passing Rate\n'

In [46]:
# use nsmallest on school summary created above to find bottom 5
bottom_performing_schools_df = school_summary_df.nsmallest(5, 'Overall Passing Rate')

In [47]:
#show dataframe
bottom_performing_schools_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",637.0,76.842711,80.744686,64.066017,77.744436,70.905226
Huang High School,District,2917,"$1,910,635.00",655.0,76.629414,81.182722,63.318478,78.813850,71.066164
Johnson High School,District,4761,"$3,094,650.00",650.0,77.072464,80.966394,63.852132,78.281874,71.067003
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.711767,81.158020,63.750424,78.433367,71.091896
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.289752,80.934412,64.746494,78.187702,71.467098


# Math Score By Grade

In [48]:
''''
Math Scores By Grade
For each school, display average math scores by grade, one grade per column
'''

def build_grade_dataframe(grade_criteria):
    '''
    Creates initial grade-level dataframes for math and reading score processing
    args:
    grade_criteria: string, the grade level to be included in dataset
    '''
    return avg_scores_by_high_school_df.loc[avg_scores_by_high_school_df['grade'] == grade_criteria]

In [49]:
#use groupby to create a dataset grouped by high school and grade, then average the columns and reset the indexes
avg_scores_by_high_school_df = students_df.groupby(['school', 'grade']).mean().reset_index()


In [50]:
#place each grade in its own dataframe
ninth_grade_df = build_grade_dataframe('9th')
tenth_grade_df = build_grade_dataframe('10th')
eleventh_grade_df = build_grade_dataframe('11th')
twelfth_grade_df = build_grade_dataframe('12th')

In [52]:
#reduce grade dataframes to school, math score, reading score
ninth_grade_red_df = pd.DataFrame({"school": ninth_grade_df["school"],
                                   "reading_score_9th": ninth_grade_df['reading_score'],
                                   "math_score_9th": ninth_grade_df['math_score']})
tenth_grade_red_df = pd.DataFrame({"school": tenth_grade_df["school"],
                                   "reading_score_10th": tenth_grade_df['reading_score'],
                                   "math_score_10th": tenth_grade_df['math_score']})
eleventh_grade_red_df = pd.DataFrame({"school": eleventh_grade_df["school"],
                                   "reading_score_11th": eleventh_grade_df['reading_score'],
                                   "math_score_11th": eleventh_grade_df['math_score']})
twelfth_grade_red_df = pd.DataFrame({"school": twelfth_grade_df["school"],
                                   "reading_score_12th": twelfth_grade_df['reading_score'],
                                   "math_score_12th": twelfth_grade_df['math_score']})
                                                                       

In [53]:
#merge the dataframes into a single dataframe
all_scores_by_grade_hs_df = pd.merge(ninth_grade_red_df, tenth_grade_red_df, on = 'school')\
.merge(eleventh_grade_red_df, on = 'school')\
.merge(twelfth_grade_red_df, on = 'school') 

In [54]:
#create new display dataframe
math_scores_by_grade_df = pd.DataFrame({'school': all_scores_by_grade_hs_df['school'],
                                        '9th': all_scores_by_grade_hs_df['math_score_9th'],
                                        '10th': all_scores_by_grade_hs_df['math_score_10th'],
                                        '11th': all_scores_by_grade_hs_df['math_score_11th'],
                                        '12th': all_scores_by_grade_hs_df['math_score_12th']})

In [55]:
#reassemble columns in display order, set index to school name and delete label
math_scores_by_grade_df = math_scores_by_grade_df[['school', '9th', '10th', '11th', '12th']]\
                 .set_index('school').rename_axis(None)

In [56]:
#display dataframe
math_scores_by_grade_df

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


# Reading Score By Grade

In [57]:
'''
Reading Scores By Grade
For each school, display average reading scores by grade, one grade per column
Uses combined math/reading score dataset created in 'Math Scores By Grade' summary
'''

"\nReading Scores By Grade\nFor each school, display average reading scores by grade, one grade per column\nUses combined math/reading score dataset created in 'Math Scores By Grade' summary\n"

In [58]:
#Create new dataframe of reading scores
reading_scores_by_grade_df = pd.DataFrame({'school': all_scores_by_grade_hs_df['school'],
                                        '9th': all_scores_by_grade_hs_df['reading_score_9th'],
                                        '10th': all_scores_by_grade_hs_df['reading_score_10th'],
                                        '11th': all_scores_by_grade_hs_df['reading_score_11th'],
                                        '12th': all_scores_by_grade_hs_df['reading_score_12th']})

In [59]:
#Reassemble columns in display order, set index to school and delete index label
reading_scores_by_grade_df = reading_scores_by_grade_df[['school', '9th', '10th', '11th', '12th']]\
                 .set_index('school').rename_axis(None)
    
#display dataframe
reading_scores_by_grade_df

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


# Score by School Spending

In [62]:
'''
Scores by School Spending
School performance by average spending per student
- Avg math and reading scores
- % passing math and reading
- overall passing rate
- categorized spending range (per student) -- >$585, $585-$615, $615-$645, $645-$675
Based on school_summary_df dataframe from 'School Summary' step
'''

"\nScores by School Spending\nSchool performance by average spending per student\n- Avg math and reading scores\n- % passing math and reading\n- overall passing rate\n- categorized spending range (per student) -- >$585, $585-$615, $615-$645, $645-$675\nBased on school_summary_df dataframe from 'School Summary' step\n"

In [98]:
#dataframe
school_summary_spending = school_summary_df.copy()
school_summary_spending

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Huang High School,District,2917,"$1,910,635.00",655.0,76.629414,81.182722,63.318478,78.813850,71.066164
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.711767,81.158020,63.750424,78.433367,71.091896
Shelton High School,Charter,1761,"$1,056,600.00",600.0,83.359455,83.725724,89.892107,92.617831,91.254969
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.289752,80.934412,64.746494,78.187702,71.467098
Griffin High School,Charter,1468,"$917,500.00",625.0,83.351499,83.816757,89.713896,93.392371,91.553134
Wilson High School,Charter,2283,"$1,319,574.00",578.0,83.274201,83.989488,90.932983,93.254490,92.093736
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.061895,83.975780,89.558665,93.864370,91.711518
Bailey High School,District,4976,"$3,124,928.00",628.0,77.048432,81.033963,64.630225,79.300643,71.965434
Holden High School,Charter,427,"$248,087.00",581.0,83.803279,83.814988,90.632319,92.740047,91.686183
Pena High School,Charter,962,"$585,858.00",609.0,83.839917,84.044699,91.683992,92.203742,91.943867


In [64]:
#create bins to hold values
bins = [0, 585, 615, 645, 675]


#Remove $ from 'Per Student Budget' and cast as a float to allow calculations using that column
school_summary_spending['Per Student Budget'] = school_summary_spending['Per Student Budget']\
.replace( '[\$]','',regex=True).astype('float')

In [72]:
#create groups for each bin
view_groups = ['>$585', '\$585-\$615', '\$615-\$645', '\$645-\$675']

In [73]:
#create new column to hold spending range category for each school
school_summary_spending['Spending Ranges (Per Student)'] = pd.cut(school_summary_spending['Per Student Budget'],
                                                                  bins, labels=view_groups)


In [74]:
#Use selected columns to build dataset for display
school_summary_spending = school_summary_spending[['Average Math Score', 'Average Reading Score',
                                          '% Passing Math', '% Passing Reading', 'Overall Passing Rate',
                                          'Spending Ranges (Per Student)']]

In [75]:
#group by Spending Range and average all scores for summary
scores_by_school_spending_df = school_summary_spending.groupby('Spending Ranges (Per Student)').mean()

In [76]:
#display dataframe
scores_by_school_spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
>$585,83.455399,83.933814,90.350436,93.325838,91.838137
\$585-\$615,83.599686,83.885211,90.788049,92.410786,91.599418
\$615-\$645,79.079225,81.891436,73.021426,83.214343,78.117884
\$645-\$675,76.997210,81.027843,63.972368,78.427809,71.200088


# Scores By School Size

In [77]:
'''
Scores by School Size
School performance by number of students
- Avg math and reading scores
- % passing math and reading
- overall passing rate
- categorized spending range (per student) -- Small(<1000), Medium(1000-2000), Large(2000-5000)
Based on school_summary_df dataframe from 'School Summary' step
'''

"\nScores by School Size\nSchool performance by number of students\n- Avg math and reading scores\n- % passing math and reading\n- overall passing rate\n- categorized spending range (per student) -- Small(<1000), Medium(1000-2000), Large(2000-5000)\nBased on school_summary_df dataframe from 'School Summary' step\n"

In [93]:
#school_summary_size is a deep copy of school_summary_df, so changes to this dataframe do not effect the original
#dataframe
school_summary_size = school_summary_df.copy()
school_summary_size


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Huang High School,District,2917,"$1,910,635.00",655.0,76.629414,81.182722,63.318478,78.813850,71.066164
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.711767,81.158020,63.750424,78.433367,71.091896
Shelton High School,Charter,1761,"$1,056,600.00",600.0,83.359455,83.725724,89.892107,92.617831,91.254969
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.289752,80.934412,64.746494,78.187702,71.467098
Griffin High School,Charter,1468,"$917,500.00",625.0,83.351499,83.816757,89.713896,93.392371,91.553134
Wilson High School,Charter,2283,"$1,319,574.00",578.0,83.274201,83.989488,90.932983,93.254490,92.093736
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.061895,83.975780,89.558665,93.864370,91.711518
Bailey High School,District,4976,"$3,124,928.00",628.0,77.048432,81.033963,64.630225,79.300643,71.965434
Holden High School,Charter,427,"$248,087.00",581.0,83.803279,83.814988,90.632319,92.740047,91.686183
Pena High School,Charter,962,"$585,858.00",609.0,83.839917,84.044699,91.683992,92.203742,91.943867


In [94]:
#create bins to hold values
bins = [0, 1000, 2000, 5000]

In [96]:
#create groups for each bin
view_groups = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']
view_groups

['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

In [81]:
#cast school size to numeric to allow for calculations
school_summary_size['Total Students'] = pd.to_numeric(school_summary_size['Total Students'])

In [83]:
#categorize school size into Small/Medium/Large bins
school_summary_size['School Size'] = pd.cut(school_summary_size['Total Students'],
                                                                  bins, labels=view_groups)

In [84]:
#Assemble selected columns for summary
school_summary_size = school_summary_size[['Average Math Score', 'Average Reading Score',
                                          '% Passing Math', '% Passing Reading', 'Overall Passing Rate',
                                          'School Size']]


In [85]:
#group by size bins and average scores
scores_by_school_size_df = school_summary_size.groupby('School Size').mean()

#display dataframe
scores_by_school_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,91.158155,92.471895,91.815025
Medium (1000-2000),83.374684,83.864438,89.931303,93.244843,91.588073
Large (2000-5000),77.746417,81.344493,67.631335,80.190800,73.911067


In [86]:
'''
Scores by School Type (District/Charter)
School performance by school type
- school type
- Avg math and reading scores
- % passing math and reading
- overall passing rate
Based on school_summary_df dataframe from 'School Summary' step
'''

"\nScores by School Type (District/Charter)\nSchool performance by school type\n- school type\n- Avg math and reading scores\n- % passing math and reading\n- overall passing rate\nBased on school_summary_df dataframe from 'School Summary' step\n"

In [87]:
#school_summary_type is a deep copy of school_summary_df, so changes to this dataframe do not effect the original
#dataframe
school_summary_type = school_summary_df.copy()

In [88]:
#assemble selected columns for display
school_summary_type = school_summary_type[['School Type','Average Math Score', 'Average Reading Score',
                                          '% Passing Math', '% Passing Reading', 'Overall Passing Rate']]

In [89]:
#group by school type and average columns
scores_by_school_type_df = school_summary_type.groupby('School Type').mean()

#display dataframe
scores_by_school_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,91.708019
District,76.956733,80.966636,64.302528,78.324559,71.313543
